# Capstone Week 3 Peer Graded Assignment
*Segmenting and Clustering Neighborhoods in Toronto*

 To create the above dataframe:
   * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
   * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
   * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
   * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.*
   * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
   * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

# 1. Get the Wiki page


In [1]:
import requests
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Get Wikipedia page is converted to a string
raw_random_wikipedia_page=requests.get(wikipedia_link)

#extract the XML as a text file a string and assign the result variable page
page = raw_random_wikipedia_page.text
#print(page)

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20181102115600-0000


# 2. Exract the postal Codes into a dataframe
*Print the shape of the dataframe and list the top 10 rows*


In [2]:
# Extracting the postal codes into dataframe
import pandas as pd
from bs4 import BeautifulSoup as bs
html_soup = bs(page,'html.parser')
#postcode_container=xml_soup.tbody.contents
#print(postcode_container)

table_rows = html_soup.find_all('tr')
#print(table_rows)
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        if str(row).startswith("['NL") or str(row).startswith("['A"):
            #print("#row")
            res
        else:
            res.append(row)
#print(res)           
        
df = pd.DataFrame(res, columns=["Postcode", "Borough", "Neighbourhood"])
print(df.shape)
print(df.head(10))
    

(289, 3)
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights
7      M6A        North York    Lawrence Manor
8      M7A      Queen's Park      Not assigned
9      M8A      Not assigned      Not assigned


# 3. Format the data frame to remove Boroughs with "Not assigned"
*Print the shape of the formatted dataframe and the top 10 rows*

In [3]:
df_formated = df[df.Borough != 'Not assigned']
print(df_formated.shape)
print(df_formated.head(10))

(212, 3)
   Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Not assigned
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern


# 4. Combine rows for similar postal code
*More than one neighborhood can exist in one postal code area*

In [4]:
df_sorted = df.sort_values(by='Postcode').reset_index()
del df_sorted['index']
#print(df_sorted.head(11))
print("************New Line************")
df_sorted = df_sorted.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
print(df_sorted.head(11))


    

************New Line************
   Postcode       Borough                                    Neighbourhood
0       M1A  Not assigned                                     Not assigned
1       M1B   Scarborough                                   Malvern, Rouge
2       M1C   Scarborough           Port Union, Rouge Hill, Highland Creek
3       M1E   Scarborough                West Hill, Morningside, Guildwood
4       M1G   Scarborough                                           Woburn
5       M1H   Scarborough                                        Cedarbrae
6       M1J   Scarborough                              Scarborough Village
7       M1K   Scarborough      East Birchmount Park, Ionview, Kennedy Park
8       M1L   Scarborough                  Oakridge, Golden Mile, Clairlea
9       M1M   Scarborough  Cliffcrest, Cliffside, Scarborough Village West
10      M1N   Scarborough                      Cliffside West, Birch Cliff


# 5. Format dataframe to update Neighbourhood value
*If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. *

In [8]:
print("************New Line************")
print(df_sorted.loc[(df_sorted['Borough'] != 'Not assigned') & (df_sorted['Neighbourhood'] == 'Not assigned')])   

for index, row in df_sorted.iterrows():
    
    if (df_sorted.loc[index,'Borough'] != 'Not assigned') & (df_sorted.loc[index,'Neighbourhood'] == 'Not assigned'):
        print("************New Line************")
        Borough = df_sorted.loc[index,'Borough']
        print('Borough: ', Borough)
        df_sorted.loc[index,'Neighbourhood'] = Borough
        #print(df_sorted)
                      
print(df_sorted.loc[(df_sorted['Postcode'] == 'M7A')])
                                                               

************New Line************
Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []
    Postcode       Borough Neighbourhood
120      M7A  Queen's Park  Queen's Park


# 6. Display the size of the dataframe
*In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe*

In [12]:
print(df_sorted.shape)

(180, 3)
